In [32]:
import torch
import pandas as pd
from tqdm import tqdm 
import torch
import time
# We used pip install transformers, pip install sentencepiece
from transformers import BertTokenizer, BertForSequenceClassification, AlbertTokenizer, AlbertForSequenceClassification
from transformers import AdamW
from transformers import T5Tokenizer, T5ForConditionalGeneration

from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
import plotly.express as px

In [2]:
data_path = 'projectData/'
tb_dir = 'tbs_HW2/'

In [3]:
#Define our device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
if torch.cuda.is_available():
    print(f'using: {torch.cuda.get_device_name(0)}')
else:
    print('using cpu')

using: NVIDIA GeForce GTX 970


### Load Data

In [4]:
raw_data = pd.read_csv(data_path+'dataset_raw.csv')

In [5]:
#Shuffle the data
raw_data = raw_data.sample(frac = 1)

In [6]:
#Create dataser class for T5
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, data ,label):
        self.data = data
        self.label = label
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
        return X, y

In [7]:
def get_data(input_df, train_size,test_size,batch_size):
    train_end_idx = train_size
    test_end_idx = train_size + test_size
    #Make one list for all the reviews
    reviews = input_df['review'].tolist()

    #mini sample of reviews for train and test
    train_data = reviews[:train_end_idx]
    test_data = reviews[train_end_idx:test_end_idx]
    #Take mini sample of the labels and preprocess them such that we can use them in the model training loop
    labels = input_df['label'].tolist()
    train_labels = labels[:train_end_idx]
    test_labels = labels[train_end_idx:test_end_idx]

    #Use data set class in order to build train and test datasets
    train_dataset = SimpleDataset(train_data, train_labels)
    test_dataset = SimpleDataset(test_data, test_labels)

    #Build train and test dataloaders
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
    return train_dataloader , test_dataloader

## Define Tokenizer and 'Bert' Model

In [8]:
#Define Bert tokenizer and model
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

#Define AlBert tokenizer and model
bert_tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
bert_model = AlbertForSequenceClassification.from_pretrained("albert-base-v2")

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [9]:
#Bert base uncased architecture
bert_model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [10]:
n_parameters = sum(p.numel() for p in bert_model.parameters())
print(f"Number of parameters in model: {n_parameters}")

Number of parameters in model: 11685122


In [11]:
def tokenizing_batch_Bert(X, y, tokenizer) :
    """
    Does ... TODO
    X: Batch of reviews 
    y: labels of reviews in batch
    """
    inputs =  tokenizer(X, max_length = 128, padding = 'max_length', truncation = True, return_tensors="pt")

    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    labels = torch.LongTensor(y).T.to(device)
    
    return input_ids, token_type_ids, attention_mask, labels

## Training Loop and Hyperparameter Selection

In [12]:
def calc_accuracy(target,pred):
    target, pred = target.to(device),pred.to(device)
    correct = torch.sum(target==pred)
    accuracy = correct/len(target)
    return accuracy

In [95]:
def fineTuning(model, optim, epochs, train_dataloader, test_dataloader, tb_dirname, tokenizer,model_type):
    train_loss_list = []
    test_loss_list = []
    train_acc_list = []
    test_acc_list = []

    writer = SummaryWriter(log_dir=f'{tb_dir}/{tb_dirname}_{time.time()}')

    for epoch in tqdm(range(epochs), desc = "epochs", position=0):
        running_loss = 0
        model.train()
        # setup loop with TQDM and dataloader
        train_loop = tqdm(train_dataloader, desc='train',position =1, leave=False)
        for X, y in train_loop:
            # initialize calculated gradients (from prev step)
            optim.zero_grad()
            # pull all tensor batches required for training
            if model_type == 'bert':
                input_ids, token_type_ids, attention_mask, labels = tokenizing_batch_Bert(X, y, tokenizer)
                outputs = model(input_ids, attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels)
                

            elif model_type == 't5':
                ids, mask, labels = tokenizing_batch_T5(X, y, tokenizer)
                outputs = model(
                input_ids=ids,
                attention_mask=mask,
                labels=labels
                )

            # process
            loss = outputs.loss
            # calculate loss for every parameter that needs grad update
            loss.backward()
            # update parameters
            optim.step()
            running_loss+=loss.item()
            # print relevant info to progress bar
            # train_loop.set_description(f'Epoch {epoch}')
            train_loop.set_postfix(loss=loss.item())
        
        epoch_train_loss = running_loss/len(train_dataloader.dataset)
        train_loss_list.append(epoch_train_loss)


        test_loss = 0
        test_acc = 0
        model.eval()
        # we dont need to update weights, so we define no_grad() to save memory
        with torch.no_grad():
            eval_loop = tqdm(test_dataloader)
            for X_test, y_test in eval_loop:
                
                if model_type == 'bert':
                    input_ids, token_type_ids, attention_mask, labels = tokenizing_batch_Bert(X_test, y_test, tokenizer)
                    outputs = model(input_ids, attention_mask=attention_mask,
                                token_type_ids=token_type_ids,
                                labels=labels)
                    # get classification
                    logits = outputs.logits
                    predicted_token_class_ids = logits.argmax(-1)
                    # calculate accuracy score
                    acc_score = calc_accuracy(y_test, predicted_token_class_ids)
                    loss = outputs.loss
                
                elif model_type == 't5':
                    ids, mask, labels = tokenizing_batch_T5(X_test, y_test, tokenizer)
                    
                    generated_ids = model.generate(
                    input_ids=ids,
                    attention_mask=mask,
                    max_length=2
                    )

                    # For printing of real words 
                    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
                    target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in labels]

                    loss = model(input_ids=ids, labels=labels).loss
                    acc_score = calc_accuracy(generated_ids[:,1], labels[:,0])
                    
                test_acc += acc_score.item()
                batch_test_loss = outputs.loss.item()
                test_loss+= batch_test_loss
                eval_loop.set_description(f'Epoch {epoch}')
                eval_loop.set_postfix(loss=batch_test_loss)
                
        epoch_acc_score = test_acc/len(test_dataloader)
        test_acc_list.append(epoch_acc_score)
        epoch_test_loss = test_loss/len(test_dataloader.dataset)
        test_loss_list.append(epoch_test_loss)

        writer.add_scalar(tag='loss/train', scalar_value=epoch_train_loss, global_step=epoch)
        writer.add_scalar(tag='loss/test', scalar_value=epoch_test_loss, global_step=epoch)
       
    
    return {
            'train_loss':train_loss_list,
            'test_loss':test_loss_list,
            'train_acc':train_acc_list,
            'test_acc':test_acc_list
            }

In [43]:
#Enter model to device
bert_model = AlbertForSequenceClassification.from_pretrained("albert-base-v2")
bert_model.to(device)
#Define optimizer
optim = torch.optim.AdamW(bert_model.parameters(), lr = 5e-5 )
#Define epochs number
epochs = 10
train_dataloader, test_dataloader = get_data(raw_data,
                                            train_size = 500,
                                            test_size = 100,
                                            batch_size = 16)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [44]:
#Fine Tuning Bert
loss_dic = fineTuning(  model=bert_model,
                        optim=optim,
                        epochs= epochs,
                        train_dataloader = train_dataloader,
                        test_dataloader=test_dataloader,
                        tb_dirname ='Bert',
                        tokenizer=bert_tokenizer,
                        model_type='bert')

Epoch 9: 100%|██████████| 7/7 [00:01<00:00,  4.55it/s, loss=0.393] 


In [47]:
a = loss_dic['test_loss']
# px.line(a)
a

[0.04674081981182098,
 0.042834992408752444,
 0.04520301282405853,
 0.04039554446935654,
 0.04883110746741295,
 0.03474756591022014,
 0.049614324271678924,
 0.041130890548229215,
 0.04317125007510185,
 0.044952907860279084]

In [102]:
torch.cuda.empty_cache()

# T5

## Preprocess


In [48]:
#Read the raw data
raw_data = pd.read_csv(data_path+'dataset_raw.csv')
#Shuffle the data
raw_data = raw_data.sample(frac = 1)
#Make copy
data_T5 = raw_data.copy()
#Mapping 0 and 1 to "positive" and "negative"
data_T5['label']=data_T5['label'].map({1: 'positive', 0: 'negative'})

In [49]:
#Adding the "sst2 sentence:" prefix for the reviews
def add_T5_preffix_sentimant_classiication(row):
    row['review'] = "sst2 sentence: " + row['review'] 
    row['label'] = row['label']
    return row

data_T5 = data_T5.apply(lambda row: add_T5_preffix_sentimant_classiication(row), axis =1)


In [50]:
#Make lists for reviews and lables
reviews = data_T5['review'].tolist()
labels  = data_T5['label'].tolist()

In [62]:
#Define T5 Tokenizer and model
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [55]:
#T5 small architecture
t5_model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [56]:
n_parameters = sum(p.numel() for p in t5_model.parameters())
print(f"Number of parameters in model: {n_parameters}")

Number of parameters in model: 60506624


In [70]:
def tokenizing_batch_T5(X, y, tokenizer) :
    train_encoder_inputs = tokenizer(X, padding="longest", max_length=128, truncation=True, return_tensors="pt")
    train_decoder_inputs = tokenizer(y, padding="longest", max_length=2, truncation=True, return_tensors="pt")

    labels = train_decoder_inputs["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    labels = labels.to(device)
    ids = train_encoder_inputs["input_ids"].to(device)
    mask = train_encoder_inputs["input_ids"].to(device)
    
    return ids, mask, labels

In [101]:
#Enter model to device
t5_model.to(device)
#Define optimizer
optim = torch.optim.AdamW(t5_model.parameters(), lr = 5e-5 )
#Define epochs number
epochs = 5
train_dataloader, test_dataloader = get_data(data_T5,
                                            train_size = 750,
                                            test_size = 250,
                                            batch_size = 16)

In [98]:
loss_dic_t5 = fineTuning(model=t5_model,
                        optim=optim,
                        epochs= epochs,
                        train_dataloader = train_dataloader,
                        test_dataloader=test_dataloader,
                        tb_dirname ='T5',
                        tokenizer=t5_tokenizer,
                        model_type='t5')

epochs: 100%|██████████| 3/3 [00:34<00:00, 11.36s/it]


In [100]:
loss_dic_t5['test_acc']

[0.0, 0.017857142857142856, 0.42857142857142855]

In [27]:

def FineTining_T5(model, optim, epochs, tokenizer , train_dataloader, test_dataloader, tb_dirname):

    train_loss_list = []
    test_loss_list = []
    test_accuracy = []

    writer = SummaryWriter(log_dir=f'{tb_dir}/{tb_dirname}_{time.time()}')


    for epoch in range(epochs):
        running_loss = 0
        model.train()
        # setup loop with TQDM and dataloader
        train_loop = tqdm(train_dataloader)
        for X, y in train_loop:
            # initialize calculated gradients (from prev step)
            optim.zero_grad()
            # pull all tensor batches required for training
            ids, mask, labels = tokenizing_batch_T5(X, y, tokenizer)
            # process
            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                labels=labels
            )
            # logits =outputs[1]
            # tokens = torch.argmax(logits, dim=2)
            # sentiments = tokenizer.batch_decode(tokens)
            # labels = tokenizer.batch_decode(y_ids)

            # extract loss
            loss = outputs[0]
            # calculate loss for every parameter that needs grad update
            loss.backward()
            # update parameters
            optim.step()
            running_loss+=loss.item()
            # print relevant info to progress bar
            train_loop.set_description(f'Epoch {epoch}')
            train_loop.set_postfix(loss=loss.item())
        
        epoch_train_loss = running_loss/len(train_dataloader.dataset)
        train_loss_list.append(epoch_train_loss)


        #test_loss = 0
        total_test_acc = 0
        model.eval()
        with torch.no_grad():
            eval_loop = tqdm(test_dataloader)
            for X_test, y_test in eval_loop:
                ids, mask, labels = tokenizing_batch_T5(X_test, y_test, tokenizer)

                generated_ids = model.generate(
                    input_ids=ids,
                    attention_mask=mask,
                    max_length=2
                    )

                preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
                target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in labels]
                print(preds)
                print(target)
                total_test_acc += accuracy_score(target, preds)
                loss = model(input_ids=ids, labels=labels).loss
            
            #epoch_test_loss = test_loss/len(test_dataloader.dataset)
            epoch_test_acc = total_test_acc / len(test_dataloader.dataset)
            #test_loss_list.append(epoch_test_loss)
            test_accuracy.append(epoch_test_acc) 

        writer.add_scalar(tag='loss/train', scalar_value=epoch_train_loss, global_step=epoch)
        #writer.add_scalar(tag='loss/test', scalar_value=epoch_test_loss, global_step=epoch)
        writer.add_scalar(tag='acc/test', scalar_value=epoch_test_acc, global_step=epoch)
    
    return {
            'train_loss':train_loss_list,
            'test_loss':test_loss_list,
            'test_accuracy': test_accuracy
            }

In [28]:
#Enter model to device
model.to(device)
#Define optimizer
optim = AdamW(model.parameters(), lr = 1e-4)
#Define number of epochs
epochs = 5

c:\Users\liort\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
training_info=FineTining_T5(model, optim, epochs, tokenizer , train_dataloader, test_dataloader, 'T5')

  1%|          | 1/125 [00:02<04:50,  2.35s/it]

['positive', 'negative', 'negative', 'negative']
['positive', 'negative', 'positive', 'positive']


  2%|▏         | 2/125 [00:04<04:12,  2.05s/it]

['positive', 'positive', 'negative', 'negative']
['negative', 'positive', 'negative', 'negative']


  2%|▏         | 3/125 [00:05<03:54,  1.92s/it]

...

['negative', 'negative', 'negative', 'positive']
['positive', 'positive', 'positive', 'positive']


100%|██████████| 125/125 [03:08<00:00,  1.50s/it]


['negative', 'negative', 'negative', 'negative']
['positive', 'positive', 'negative', 'negative']


Epoch 3:  91%|█████████ | 228/250 [24:00<02:18,  6.32s/it, loss=0.312]


KeyboardInterrupt: 

In [ ]:
training_info

{'train_loss': [0.1478367365965205, 0.0010359602420539887],
 'test_loss': [0.0, 0.0],
 'test_accuracy': [0.0, 0.0]}